# Data preprocessing steps

In [145]:
# importing libraries

import pandas as pd
import numpy as np

In [152]:
# loading data and handling the NaNs

df = pd.read_csv('task_data.csv', na_values=r'\N', dtype={12: str, 13: str})

In [ ]:
df.loc[df['CANCELED_DATE'].notnull()]

In [155]:
# some  values in cancelled orders contain "\\N', replacing it with zero

df = df.replace(r'\\N', 0)

In [156]:
# changing data types

df['CANCELED_DATE'] = pd.to_datetime(df['CANCELED_DATE'])
df['PAYMENT_DATE'] = pd.to_datetime(df['PAYMENT_DATE'])
df['GRADED_DATE'] = pd.to_datetime(df['GRADED_DATE'])
df['SENDING_DATE'] = pd.to_datetime(df['SENDING_DATE'])
df['PURCHASED_DATE'] = pd.to_datetime(df['PURCHASED_DATE'])
df['RETURNED_DATE'] = pd.to_datetime(df['RETURNED_DATE'])
df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'])
df['REVENUE'] = pd.to_numeric(df['REVENUE'])
df.dtypes

ORDER_UUID                    object
ORDER_ITEM_ID                  int64
ITEM_ID                        int64
PRODUCT_ID                     int64
NAME                          object
CE_MEDIA                      object
MAIN_CATEGORY                 object
VARIANT_LABEL                 object
CHANNEL                       object
TYPE                          object
CREATED_DATE          datetime64[ns]
CANCELED_DATE         datetime64[ns]
PAYMENT_DATE          datetime64[ns]
GRADED_DATE           datetime64[ns]
SENDING_DATE          datetime64[ns]
PURCHASED_DATE        datetime64[ns]
RETURNED_DATE         datetime64[ns]
ORDER_RANK                     int64
NEW_CUSTOMER_ORDER            object
REVENUE                        int64
dtype: object

In [158]:
# creating new column with revenue in euros

df['amount'] = df['REVENUE'] / 100

In [138]:
# cancelled items may affect KPIs and calculations, I will save cancelled items to a separate dataset

cancelled = df.loc[df['CANCELED_DATE'].notnull()]

In [139]:
# removing cancelled orders from dataset

df = df[df['CANCELED_DATE'].isnull()]

In [78]:
cancelled.to_csv('cleared_cancelled.csv', index=False)

In [ ]:
# checking categorical values for consistency

print(df['CE_MEDIA'].value_counts())
print(df['MAIN_CATEGORY'].value_counts())
print(df['VARIANT_LABEL'].value_counts())
print(df['CHANNEL'].value_counts())
print(df['TYPE'].value_counts())
print(df['NEW_CUSTOMER_ORDER'].value_counts())

In [159]:
df['NEW_CUSTOMER_ORDER'] = df['NEW_CUSTOMER_ORDER'].str.replace('0', 'false')

In [81]:
# checking for duplicates
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [ORDER_UUID, ORDER_ITEM_ID, ITEM_ID, PRODUCT_ID, NAME, CE_MEDIA, MAIN_CATEGORY, VARIANT_LABEL, CHANNEL, TYPE, CREATED_DATE, CANCELED_DATE, PAYMENT_DATE, GRADED_DATE, SENDING_DATE, PURCHASED_DATE, RETURNED_DATE, ORDER_RANK, NEW_CUSTOMER_ORDER, REVENUE, amount]
Index: []

[0 rows x 21 columns]


In [161]:
# Get the PURCHASED_DATE for each item_id where type == 'purchase'
purchase_dates = df[df['TYPE'] == 'purchase'].set_index('ITEM_ID')['PURCHASED_DATE']

# Map the PURCHASED_DATE to sale rows
df['purchase_date_for_sale'] = df['ITEM_ID'].map(purchase_dates)

In [163]:
# Sort by item_id and CREATED_DATE
df = df.sort_values(by=['ITEM_ID', 'CREATED_DATE'])

# Get the previous RETURNED_DATE for each item_id
df['previous_returned_date'] = df.groupby('ITEM_ID')['RETURNED_DATE'].shift()

In [170]:
# Get the 'amount' for the 'purchase' transaction for each item_id
purchase_amounts = df[df['TYPE'] == 'purchase'].set_index('ITEM_ID')['amount']

# Map the 'purchase' amount to sale rows
df['purchase_amount_for_sale'] = df['ITEM_ID'].map(purchase_amounts)

# Calculate the amount difference for 'sale' rows where RETURNED_DATE is null
df['margin'] = df.apply(
    lambda row: row['amount'] - row['purchase_amount_for_sale']
    if row['TYPE'] == 'sale' and pd.isnull(row['RETURNED_DATE']) else None,
    axis=1
)

In [171]:
# checking if everything was assigned correctly

df[df['ITEM_ID'] == 70583611]

,ORDER_UUID,ORDER_ITEM_ID,ITEM_ID,PRODUCT_ID,NAME,CE_MEDIA,MAIN_CATEGORY,VARIANT_LABEL,CHANNEL,TYPE,...,PURCHASED_DATE,RETURNED_DATE,ORDER_RANK,NEW_CUSTOMER_ORDER,REVENUE,amount,purchase_date_for_sale,previous_returned_date,purchase_amount_for_sale,margin
717567,1e74207f-45f8-43e0-a115-547b0c33d210,130612440,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,A1,B2B,purchase,...,2017-12-15,NaT,11,false,44784,447.84,2017-12-15,NaT,447.84,NaN
761630,2287fb91-3000-477a-8337-9647fb8daa83,138562397,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,A1,reBuy.de,sale,...,NaT,2018-03-29,1,true,36942,369.42,2017-12-15,NaT,447.84,NaN
794944,525d6a03-a7b7-4fc4-9757-882a7e882c5a,144596878,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,A1,reBuy.de,sale,...,NaT,2018-08-27,1,true,48821,488.21,2017-12-15,2018-03-29,447.84,NaN
894645,86d2538a-dae2-4d85-8038-b3b056a456c7,162595867,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,No Variant,ebay,sale,...,NaT,2019-03-06,11,false,4389,43.89,2017-12-15,2018-08-27,447.84,NaN
910691,9eefc533-8ea5-49ce-ba21-d6edfc5424f1,165542184,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,No Variant,ebay,sale,...,NaT,2019-04-09,2,false,8290,82.90,2017-12-15,2019-03-06,447.84,NaN
921604,94b6c984-5ff8-4451-957f-da8362ab6b30,167480887,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,No Variant,ebay,sale,...,NaT,2019-04-25,63,false,13448,134.48,2017-12-15,2019-04-09,447.84,NaN
932150,532762c8-53ea-4c56-9775-caf1419ac1d8,169384600,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,No Variant,ebay,sale,...,NaT,2019-05-15,76,false,3714,37.14,2017-12-15,2019-04-25,447.84,NaN
939697,59d22adf-39d3-421b-a951-063b6406524d,170750310,70583611,10274582,Apple iPhone 6s 64GB roségold,CE,Handy,No Variant,ebay,sale,...,NaT,NaT,7,false,11082,110.82,2017-12-15,2019-05-15,447.84,-337.02


In [172]:
last_year = df[df['CREATED_DATE'] >= '2018-01-01']

In [173]:
last_year = last_year.drop('CANCELED_DATE', axis=1)

In [175]:
last_year.to_csv('last_years.csv', index=False)